In [1]:
# Dataset Preparation
"""
- get the image from internet or public dataset like roboflow etc
- annotate every images to get their object which strawberry in every images and save in csv or txt file
    - for my case, i took format annotate as [ filename, width_image, height_image, class_object, xmin, ymin, xmax, ymax]
    - convert csv or txt with annotated images and images to tfrecord format
"""
# Processing which modelling, training, save model, for optional convert to tflite with or without quantized
"""
- import objec

"""

'\n- import objec\n\n'

In [ ]:
# get tensorflow object_detection API 
!git clone --depth 1 https://github.com/tensorflow/models.git

Cloning into 'models'...
error: RPC failed; curl 92 HTTP/2 stream 5 was not closed cleanly: CANCEL (err 8)
error: 6489 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output


In [ ]:
# For compatibility. Pin tf-models-official version so it will use Tensorflow 2.15.

!sed -i 's/tf-models-official>=2.5.1/tf-models-official==2.15.0/g' ./models/research/object_detection/packages/tf2/setup.py

# Compile the Object Detection API protocol buffers and install the necessary packages which in setup.py
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

In [2]:
import tensorflow as tf
import numpy as np
import object_detection
import matplotlib.pyplot as plt

from object_detection.utils import config_util, label_map_util, dataset_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format


In [9]:
# i use python 3.10 and compatible too for 3.11 for this case
# tensorflow 2.15

In [10]:
# adjust to yout needs
model_dir = '/content/my_model'

In [ ]:
# Download pretarined model SSDMobilenetV2FPNLite with input shape imgae 320x320
!wget -- 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

# move model to yout directory and tar it, i use G.colab and move it in 'sample_data' dir. 
!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {'sample_data'}
!cd {'sample_data'} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

In [ ]:
#copy pipeline.config to your model dir
!cp /content/sample_data/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config /content/my_model/

In [ ]:
# check the config
config_path = '/content/my_model/pipeline.config'
configs = config_util.get_configs_from_pipeline_file(config_path)

In [ ]:
# you can manually adjust the config with open them, in this i use code for adjuts a little bit
"""
- num classes from 90 to 3 for my case strawberry ripe [ripe, half ripe, not ripe]
- checkpoint type into detection
- set checkpoint to pretrained model checkpoint
- and set my dataset which train, valid and label
"""

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

pipeline_config.model.ssd.num_classes = 3
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint ='/content/sample_data/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= '/content/my_model/label.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = ['/content/my_model/Train.tfrecord']
pipeline_config.eval_input_reader[0].label_map_path = '/content/my_model/label.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = ['/content/my_model/Valid.tfrecord']


In [ ]:
#check pipeline
pipeline_config

In [ ]:
# set config
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(config_path, "wb") as f:
    f.write(config_text)
configs = config_util.get_configs_from_pipeline_file(config_path)

In [ ]:
# for my case which use tensorflow 2.15, some code in decoder.py needs to be changed

# Path to file tfexample_decoder.py
dirr_pack = '/usr/local/lib/python3.10/dist-packages'
file_path = f"{dirr_pack}/tf_slim/data/tfexample_decoder.py"


# Membaca dan memodifikasi file
with open(file_path, 'r') as f:
    lines = f.readlines()

# Memodifikasi konten file
with open(file_path, 'w') as f:
    for i, line in enumerate(lines):
        if i == 27:
            f.write("import tensorflow as tf\n")
        
        if "control_flow_ops.case" in line:
            line = line.replace("control_flow_ops.case", "tf.case")
        f.write(line)

        if "control_flow_ops.cond" in line:
            line = line.replace("control_flow_ops.cond", "tf.cond")
        f.write(line)

print("finish")


In [11]:
# train model with model_main_tf2.py
"""
- set pipeline config to your pipeline.config.file
- set model_dir to your model directory, the checkpoint, train info, and train result would be in there
- set num train, in my case i set to 20,000 times instead of 50,000 for default
"""

!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=/content/my_model/pipeline.config\
    --model_dir=/content/my_model \
    --num_train_steps=20000 \
    --sample_1_of_n_eval_examples=1 \
    --alsologtostderr

python: can't open file 'c:\\content\\models\\research\\object_detection\\model_main_tf2.py': [Errno 2] No such file or directory


In [4]:
# Path to the checkpoint directory
checkpoint_dir = '/content/my_model'
configs = config_util.get_configs_from_pipeline_file(f'{checkpoint_dir}/pipeline.config')

# Load pipeline config and build detection model

model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(f'{checkpoint_dir}/ckpt-21').expect_partial()

export = 'model/1/'
tf.saved_model.save(detection_model, export)

INFO:tensorflow:Assets written to: model/1/assets


INFO:tensorflow:Assets written to: model/1/assets


In [6]:
import app
app


<module 'app' from 'c:\\Users\\hafid\\Downloads\\Strawberry_Detection_with_SSDMobilenetV2\\app.py'>